In [1]:
# train a 3 layer MLP on mnist
import os
# os.environ["CUDA_VISIBLE_DEVICES"] = "5"
import torch
import torch.nn as nn
import torchvision 
from torchvision import transforms
import numpy as np 
import tqdm

model = nn.Sequential(
    nn.Linear(784, 150, bias=False),
    nn.ReLU(),
    nn.Linear(150, 200, bias=False),
    nn.ReLU(),
    nn.Linear(200, 10, bias=False),
)

if torch.__version__.startswith("2"):
    print("Using torch 2.0 compile")
    torch.compile(model)
    
train_set = torchvision.datasets.MNIST(
    root='./data',
    train=True,
    download=True,
    transform=transforms.Compose([
        transforms.RandomRotation(10),
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        torch.flatten
    ])
)
test_set = torchvision.datasets.MNIST(
    root='./data',
    train=False,
    download=True,
    transform=transforms.Compose([
        transforms.
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,)),
        torch.flatten
    ])
)

train_loader = torch.utils.data.DataLoader(
    dataset=train_set,
    batch_size=128,
    shuffle=True
)
test_loader = torch.utils.data.DataLoader(
    dataset=test_set,
    batch_size=128,
    shuffle=True
)

loss_fn = nn.CrossEntropyLoss()
optim = torch.optim.SGD(model.parameters(), lr=0.1, momentum=0.9, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.ReduceLROnPlateau(
    optim,
    mode='max',
    factor=0.1,
    patience=2,
    min_lr = 1e-4
)


epochs = 12
eval_every = 100 # batches 

best_model = None

with torch.autocast("cuda"):
    for epoch in tqdm.tqdm(range(epochs)):
        
        train_loss = 0
        eval_loss = 0
        train_acc = 0
        eval_acc = 0
        best_acc = 0
        
        # train
        for i, (x, y) in enumerate(train_loader):
            y_pred = model(x)
            loss = loss_fn(y_pred, y)
            optim.zero_grad()
            loss.backward()
            optim.step()
            
            acc = (y_pred.argmax(dim=1) == y).float().mean()
            train_acc += acc 
            train_loss += loss.item()
            
            # eval
            if i % eval_every == 0:
                for x, y in test_loader:
                    model.eval()
                    y_pred = model(x)
                    loss = loss_fn(y_pred, y)
                    
                    eval_loss += loss.item()
                    acc = (y_pred.argmax(dim=1) == y).float().mean()
                    eval_acc += acc    
                    
                print(f"Epoch {epoch}: train acc: {train_acc / eval_every}, train loss: {train_loss / eval_every},"
                    f" eval acc: {eval_acc / len(test_loader)}, eval loss: {eval_loss / len(test_loader)},"
                    f" lr: {optim.param_groups[0]['lr']}")

                
                if eval_acc / len(test_loader) > best_acc:
                    best_acc = eval_acc / len(test_loader)
                    best_model = model.state_dict()
                    
                train_loss = 0
                eval_loss = 0
                train_acc = 0
                eval_acc = 0
        
        scheduler.step(eval_acc / len(test_loader))

Using torch 2.0 compile


  0%|          | 0/12 [00:00<?, ?it/s]

Epoch 0: train acc: 0.0006249999860301614, train loss: 0.023246397972106935, eval acc: 0.10146360844373703, eval loss: 2.301863613007944, lr: 0.1
Epoch 0: train acc: 0.8145312666893005, train loss: 0.6028286927938461, eval acc: 0.9208860993385315, eval loss: 0.2596525170757801, lr: 0.1
Epoch 0: train acc: 0.9255468845367432, train loss: 0.24776186376810075, eval acc: 0.9344343543052673, eval loss: 0.20644960184640523, lr: 0.1
Epoch 0: train acc: 0.9366406202316284, train loss: 0.20699586383998395, eval acc: 0.9401701092720032, eval loss: 0.19996813892186444, lr: 0.1
Epoch 0: train acc: 0.9435155987739563, train loss: 0.18499109778553247, eval acc: 0.9466969966888428, eval loss: 0.1583833569967294, lr: 0.1


  8%|▊         | 1/12 [00:20<03:50, 20.95s/it]

Epoch 1: train acc: 0.009765625, train loss: 0.0006559369713068008, eval acc: 0.9614319801330566, eval loss: 0.12200899863073343, lr: 0.1
Epoch 1: train acc: 0.9572656154632568, train loss: 0.14233654592186212, eval acc: 0.9590585231781006, eval loss: 0.1340248633789111, lr: 0.1
Epoch 1: train acc: 0.9573437571525574, train loss: 0.14094047885388136, eval acc: 0.962124228477478, eval loss: 0.12388285706880726, lr: 0.1
Epoch 1: train acc: 0.9614843726158142, train loss: 0.12964561261236668, eval acc: 0.9670688509941101, eval loss: 0.10503417272356493, lr: 0.1
Epoch 1: train acc: 0.9625781178474426, train loss: 0.12707419376820325, eval acc: 0.962124228477478, eval loss: 0.1154981301743773, lr: 0.1


...

 83%|████████▎ | 10/12 [03:26<00:41, 20.69s/it]

Epoch 10: train acc: 0.009921874850988388, train loss: 0.00045234546065330505, eval acc: 0.9832871556282043, eval loss: 0.0593593227264436, lr: 0.00010000000000000003
Epoch 10: train acc: 0.9917187690734863, train loss: 0.027876479031983762, eval acc: 0.9839794039726257, eval loss: 0.053309094955440305, lr: 0.00010000000000000003
Epoch 10: train acc: 0.9915624856948853, train loss: 0.02765995785361156, eval acc: 0.983188271522522, eval loss: 0.059383448561111205, lr: 0.00010000000000000003
Epoch 10: train acc: 0.9924218654632568, train loss: 0.025830444670282303, eval acc: 0.9840783476829529, eval loss: 0.053318472414077084, lr: 0.00010000000000000003
Epoch 10: train acc: 0.9906250238418579, train loss: 0.029232904515229165, eval acc: 0.9832871556282043, eval loss: 0.054444220718703695, lr: 0.00010000000000000003


 92%|█████████▏| 11/12 [03:47<00:20, 20.69s/it]

Epoch 11: train acc: 0.009921874850988388, train loss: 0.00028110474348068236, eval acc: 0.9839794039726257, eval loss: 0.05329462136451414, lr: 0.00010000000000000003
Epoch 11: train acc: 0.9911718964576721, train loss: 0.029427686266135424, eval acc: 0.9839794039726257, eval loss: 0.053310558991211975, lr: 0.00010000000000000003
Epoch 11: train acc: 0.9925000071525574, train loss: 0.026688256522174925, eval acc: 0.9839794039726257, eval loss: 0.053309991943231445, lr: 0.00010000000000000003
Epoch 11: train acc: 0.9909374713897705, train loss: 0.029677310059778392, eval acc: 0.9832871556282043, eval loss: 0.05461381115549822, lr: 0.00010000000000000003
Epoch 11: train acc: 0.991406261920929, train loss: 0.026521230812650173, eval acc: 0.9832871556282043, eval loss: 0.05499958628759118, lr: 0.00010000000000000003


100%|██████████| 12/12 [04:08<00:00, 20.70s/it]


In [ ]:
# save the first 5000 images in the test set as numpy array
test_images = []
test_labels = []
for i, (x, y) in enumerate(test_loader):
    test_images.append(x)
    test_labels.append(y)

test_images = torch.cat(test_images).numpy()
test_labels = torch.cat(test_labels).numpy()
np.save("./data/test_images.npy", test_images)
np.save("./data/test_labels.npy", test_labels)

# save model weights as numpy array
for idx, (name, param) in enumerate(best_model.items()):
    name = name.split(".")
    name[0] = str(idx)
    name = ".".join(name)
    np.save(f"./data/linear_{name}.npy", param.detach().cpu().numpy()) # W @ X -> X @ W.T, cnpy reads column-major
    


((784, 150), dtype('float32'))

In [ ]:
param = np.load("./data/linear_0.weight.npy")
print("shape:", param.shape, ",", param.dtype, ", first 3 values (cupy): ", param[0, :3])

shape: (784, 150) , float32 , first 3 values:  [-0.02242141 -0.01722893  0.01018926]
